In [ ]:
#Christopher Newbolt
import pandas as pd
import numpy as np
import os
import json
import numpy as np

In [ ]:
filepath1 = os.path.join('Resources','purchase_data.json')
filepath2 = os.path.join('Resources', 'purchase_data2.json')

In [ ]:
df1 = pd.read_json(filepath1)
df2 = pd.read_json(filepath2)

In [ ]:
pur_data = pd.merge(df1,df2, how='outer')
pur_data.head()

In [ ]:
#Counts individuals 
Player_counter = len(pur_data['SN'].unique())
players_df = pd.DataFrame([{'Total Players': Player_counter}])
players_df.set_index('Total Players', inplace = True)
players_df

In [ ]:
#Removing Duplicates
remove_Dup = pur_data.drop_duplicates(['Item ID'], keep = 'last')

#counts items by unique ID
total_unique = len(remove_Dup)
#uses count.price to find total purchase
total_pur = pur_data['Price'].count()

#calculates total revenue for table by summing occurance of price and below calc
total_rev = round(pur_data['Price'].sum(),2)
#calculates total_rev
avg_price = round(total_rev/total_pur, 2)

#creates Purchase Analysis DataFrame
pur_analysis = pd.DataFrame([{
    
    'Total Purchases': total_pur,
    'Number of Unique Items': total_unique,
    'Average Purchase Price': avg_price,
    'Total Revenue': total_rev
}])

#Purchase Analysis
pur_analysis.style.format({'Average Purchase Price': '${:.2f}', 'Total Revenue': '${:,.2f}'})


In [ ]:
"""
 Gender Demographics
 Percentage and Count of Male Players
 Percentage and Count of Female Players
 Percentage and Count of Other / Non-Disclosed
"""
#creates df with no duplicates
no_dup_players = pur_data.drop_duplicates(['SN'], keep ='last')

#counts gender values 
gender_counts = no_dup_players['Gender'].value_counts().reset_index()

#adds column for % of players using player count from first table and gender_count column

gender_counts['% of Players'] = gender_counts['Gender']/Player_counter * 100
#renames columns

gender_counts.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)
#sets index as Gender 
gender_counts.set_index(['Gender'], inplace = True)

#formats table
gender_counts.style.format({"% of Players": "{:.1f}%"})

In [ ]:
""" 
 Purchasing Analysis (Gender)
 The below each broken by gender
 Purchase Count
 Average Purchase Price
 Total Purchase Value
 Normalized Totals
"""
# counts purchases by gender
pur_count_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Gender'].count())
# sums price by gender
total_pur_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Price'].sum())
#merges the two data frames from above
pur_analysis_gen = pd.merge(pur_count_by_gen, total_pur_by_gen, left_index = True, right_index = True)
#renames columns

pur_analysis_gen.rename(columns = {'Gender': '# of Purchases', 'Price':'Total Purchase Value'}, inplace=True)
#adds column for average purchase price by gender by dividing total purcahse value by gender by # of purchases by gender
pur_analysis_gen['Average Purchase Price'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Purchases']
#merges gender counts from above table (excluding dup SNs) into current df 
pur_analysis_gen = pur_analysis_gen.merge(gender_counts, left_index = True, right_index = True)
# calculates and adds normalized total column by dividing total purchase value by unique # of players by genger
pur_analysis_gen['Normalized Totals'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Players']
pur_analysis_gen

#deletes columns 
del pur_analysis_gen['% of Players']
del pur_analysis_gen['# of Players']


pur_analysis_gen.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals': '${:.2f}'})

In [ ]:
"""
The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
 Purchase Count
 Average Purchase Price
 Total Purchase Value
 Normalized Totals
"""


pur_data.loc[(pur_data['Age'] < 10), 'age_bin'] = "< 10"
pur_data.loc[(pur_data['Age'] >= 10) & (pur_data['Age'] <= 14), 'age_bin'] = "10 - 14"
pur_data.loc[(pur_data['Age'] >= 15) & (pur_data['Age'] <= 19), 'age_bin'] = "15 - 19"
pur_data.loc[(pur_data['Age'] >= 20) & (pur_data['Age'] <= 24), 'age_bin'] = "20 - 24"
pur_data.loc[(pur_data['Age'] >= 25) & (pur_data['Age'] <= 29), 'age_bin'] = "25 - 29"
pur_data.loc[(pur_data['Age'] >= 30) & (pur_data['Age'] <= 34), 'age_bin'] = "30 - 34"
pur_data.loc[(pur_data['Age'] >= 35) & (pur_data['Age'] <= 39), 'age_bin'] = "35 - 39"
pur_data.loc[(pur_data['Age'] >= 40), 'age_bin'] = "> 40"



# counts total purchases by age bin by counting screen names
pur_count_age = pd.DataFrame(pur_data.groupby('age_bin')['SN'].count())

#finds avg price of purchases by age bin
avg_price_age = pd.DataFrame(pur_data.groupby('age_bin')['Price'].mean())
#finds total purchase value by age bin
tot_pur_age = pd.DataFrame(pur_data.groupby('age_bin')['Price'].sum())
#deletes multiple occurances of SN while only keeping last, then counts # of unique
#players by age bin
no_dup_age = pd.DataFrame(pur_data.drop_duplicates('SN', keep = 'last').groupby('age_bin')['SN'].count())


#merges all info from above into one df

merge_age = pd.merge(pur_count_age, avg_price_age, left_index = True, right_index = True).merge(tot_pur_age, left_index = True, right_index = True).merge(no_dup_age, left_index = True, right_index = True)
#renames columns
merge_age.rename(columns = {"SN_x": "# of Purchases", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "SN_y": "# of Purchasers"}, inplace = True)
#ormalized totals
merge_age['Normalized Totals'] = merge_age['Total Purchase Value']/merge_age['# of Purchasers']
merge_age.index.rename("Age", inplace = True)

# formats
merge_age.style.format({'Average Purchase Price': '${:.2f}', 'Total Purchase Value': '${:.2f}', 'Normalized Totals': '${:.2f}'})